In [1]:
import random
import numpy

def set_seed(seed: int, n_gpu: int):
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

In [2]:
reviews_train = []
for line in open("../data/imdb/full_train.txt", "r"):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open("../data/imdb/full_test.txt", "r"):
    reviews_test.append(line.strip())

In [3]:
train_target = [1 if i < 12500 else 0 for i in range(25000)]
test_target = [1 if i < 12500 else 0 for i in range(25000)]

In [4]:
from itertools import chain

train_tokens = list(chain(*[[token for token in sample.lower().split()] for sample in reviews_train]))

In [5]:
from collections import Counter

train_vocabulary = Counter(train_tokens)

In [6]:
UNKNOWN_TOKEN = "unknown"
PAD_TOKEN = "PAD"

index_to_token = [UNKNOWN_TOKEN] + [PAD_TOKEN] + list(train_vocabulary.keys())
token_to_index = {token: index + 2 for index, token in enumerate(train_vocabulary.keys())}
token_to_index[UNKNOWN_TOKEN] = 0
token_to_index[PAD_TOKEN] = 1

In [7]:
EMBEDDING_DIM = 100
BATCH_SIZE = 50
MAX_INPUT_LENGTH = 100

In [8]:
def generate_batch(input_data, max_length = MAX_INPUT_LENGTH):
    texts = torch.tensor([padding(sample["text"], max_length) for sample in input_data], dtype=torch.long)
    labels = torch.tensor([sample["label"] for sample in input_data], dtype=torch.long)
    return texts, torch.tensor([]), labels

def padding(text_tokens, max_length, padding_token=1):
    if len(text_tokens) >= max_length:
        return text_tokens[:max_length]
    return text_tokens + [padding_token]*(max_length - len(text_tokens))

In [9]:
prepared_data = []

for label, text in zip(train_target, reviews_train):
    text_tokens = [token_to_index[token.lower()] for token in text.split()]
    prepared_data.append({"label": label, "text": text_tokens})

In [36]:
from torch.utils.data import DataLoader

def train(input_data, collate_fn):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(input_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    for i, (text, sequence_length, label) in enumerate(data):
        optimizer.zero_grad()
        text, sequence_length, label = text.to(device), sequence_length.to(device), label.to(device)
        output = model(text, sequence_length)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == label).sum().item()

    return train_loss / len(input_data), train_acc / len(input_data)

def test(input_data, collate_fn):
    loss = 0
    acc = 0
    data = DataLoader(input_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for i, (text, sequence_length, label) in enumerate(data):
        text, sequence_length, label = text.to(device),  sequence_length.to(device), label.to(device)
        with torch.no_grad():
            output = model(text, sequence_length)
            loss = criterion(output, label)
            loss += loss.item()
            acc += (output.argmax(1) == label).sum().item()

    return loss / len(input_data), acc / len(input_data)

## Simple RNN

In [11]:
RNN_HIDDEN_SIZE = 100
IS_BIDIRECTIONAL = False
NUM_LAYERS = 1

In [13]:
import torch
import torch.nn as nn

class SentimentClassificationSimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size, is_bidirectional, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(input_size=embed_dim, hidden_size=hidden_size, num_layers=num_layers, bidirectional=is_bidirectional)
        self.fc = nn.Linear(hidden_size*(1 + int(is_bidirectional)), num_class)
        self.init_weights()

    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

    def forward(self, text, sequence_length=None):
        embedded = self.embedding(text)
        output, _ = self.rnn(embedded)
        pooled_output, _ = output.max(dim=1)
        return self.fc(pooled_output)

In [14]:
set_seed(42, 1)

In [15]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = SentimentClassificationSimpleRNNModel(
    vocab_size=len(index_to_token), 
    embed_dim=EMBEDDING_DIM, 
    num_class=len(set(train_target)), 
    hidden_size=RNN_HIDDEN_SIZE,
    is_bidirectional=IS_BIDIRECTIONAL,
    num_layers=NUM_LAYERS
).to(device)

In [16]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 10
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

train_len = int(len(prepared_data) * 0.95)
train_data, validation_data = \
    random_split(prepared_data, [train_len, len(prepared_data) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train(train_data, generate_batch)
    valid_loss, valid_acc = test(validation_data, generate_batch)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 3 seconds
	Loss: 0.0130(train)	|	Acc: 61.7%(train)
	Loss: 0.0010(valid)	|	Acc: 65.6%(valid)
Epoch: 2  | time in 0 minutes, 3 seconds
	Loss: 0.0113(train)	|	Acc: 70.0%(train)
	Loss: 0.0009(valid)	|	Acc: 69.2%(valid)
Epoch: 3  | time in 0 minutes, 3 seconds
	Loss: 0.0105(train)	|	Acc: 73.5%(train)
	Loss: 0.0008(valid)	|	Acc: 74.2%(valid)
Epoch: 4  | time in 0 minutes, 3 seconds
	Loss: 0.0099(train)	|	Acc: 75.4%(train)
	Loss: 0.0008(valid)	|	Acc: 75.3%(valid)
Epoch: 5  | time in 0 minutes, 3 seconds
	Loss: 0.0095(train)	|	Acc: 77.0%(train)
	Loss: 0.0009(valid)	|	Acc: 72.8%(valid)
Epoch: 6  | time in 0 minutes, 3 seconds
	Loss: 0.0091(train)	|	Acc: 78.3%(train)
	Loss: 0.0008(valid)	|	Acc: 77.2%(valid)
Epoch: 7  | time in 0 minutes, 3 seconds
	Loss: 0.0088(train)	|	Acc: 79.2%(train)
	Loss: 0.0008(valid)	|	Acc: 77.8%(valid)
Epoch: 8  | time in 0 minutes, 3 seconds
	Loss: 0.0085(train)	|	Acc: 80.0%(train)
	Loss: 0.0009(valid)	|	Acc: 77.4%(valid)
Epoch: 9  | time

## Masked Padding

We have two sentences: "I like watching movies" and "I don't like watching movies". After tokenization we'll get: `["I", "like", "watching", "movies"]` (4 tokens) and `["I", "don't", "like", "watching", "movies"]` (5 tokens).

To create a batch, both sentences should have the same number of tokens, so we add to the first sentence "padding token" and get
`["I", "like", "watching", "movies", "PAD"]` (5 tokens) and `["I", "don't", "like", "watching", "movies"]` (5 tokens)

Now our model will know, that we used padding for the first sentence.
Should our model know about this?

We can mask tokens and our model will ignore them. Use `torch.nn.utils.rnn.pack_padded_sequence`

In [30]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class SentimentClassificationMaskedRNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size, is_bidirectional, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(input_size=embed_dim, hidden_size=hidden_size, num_layers=num_layers, bidirectional=is_bidirectional)
        self.fc = nn.Linear(hidden_size*(1 + int(is_bidirectional)), num_class)
        self.init_weights()

    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

    def forward(self, text, sequence_length):
        embedded = self.embedding(text)
        packed_output, _ = self.rnn(pack_padded_sequence(embedded, sequence_length, batch_first=True))
        output, _ = pad_packed_sequence(packed_output)
        pooled_output, _ = output.max(dim=0)
        return self.fc(pooled_output)

In [31]:
def generate_batch_with_masking(input_data, max_length = MAX_INPUT_LENGTH):
    # return (text, sequence_length), labels
    prepared_data = []
    sequence_length = []
    for sample in input_data:
        prepared_data.append(padding(sample["text"], max_length))
        sequence_length.append(min(len(sample["text"]), max_length))
        
    prepared_data = torch.tensor(prepared_data, dtype=torch.long)
    sequence_length = torch.tensor(sequence_length, dtype=torch.long)
    sequence_length, perm_idx = sequence_length.sort(0, descending=True)
    prepared_data = prepared_data[perm_idx]
    
    labels = torch.tensor([sample["label"] for sample in input_data], dtype=torch.long)[perm_idx]
    return prepared_data, sequence_length, labels

In [32]:
generate_batch_with_masking(
    [
        {"text": [1, 2, 3, 4], "label": 1}, 
        {"text": [1, 5, 2, 3, 4], "label": 0}, 
    ]
)

(tensor([[1, 5, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1],
         [1, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1]]),
 tensor([5, 4]),
 tensor([0, 1]))

In [33]:
set_seed(42, 1)

In [34]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = SentimentClassificationMaskedRNNModel(
    vocab_size=len(index_to_token), 
    embed_dim=EMBEDDING_DIM, 
    num_class=len(set(train_target)), 
    hidden_size=RNN_HIDDEN_SIZE,
    is_bidirectional=IS_BIDIRECTIONAL,
    num_layers=NUM_LAYERS
).to(device)

In [37]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 10
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

train_len = int(len(prepared_data) * 0.95)
train_data, validation_data = \
    random_split(prepared_data, [train_len, len(prepared_data) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train(train_data, generate_batch_with_masking)
    valid_loss, valid_acc = test(validation_data, generate_batch_with_masking)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 6 seconds
	Loss: 0.0108(train)	|	Acc: 72.1%(train)
	Loss: 0.0007(valid)	|	Acc: 72.7%(valid)
Epoch: 2  | time in 0 minutes, 5 seconds
	Loss: 0.0097(train)	|	Acc: 76.1%(train)
	Loss: 0.0007(valid)	|	Acc: 71.8%(valid)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0090(train)	|	Acc: 78.7%(train)
	Loss: 0.0006(valid)	|	Acc: 74.0%(valid)
Epoch: 4  | time in 0 minutes, 5 seconds
	Loss: 0.0084(train)	|	Acc: 80.5%(train)
	Loss: 0.0007(valid)	|	Acc: 75.1%(valid)
Epoch: 5  | time in 0 minutes, 5 seconds
	Loss: 0.0078(train)	|	Acc: 82.0%(train)
	Loss: 0.0006(valid)	|	Acc: 76.4%(valid)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0072(train)	|	Acc: 84.0%(train)
	Loss: 0.0005(valid)	|	Acc: 77.1%(valid)
Epoch: 7  | time in 0 minutes, 6 seconds
	Loss: 0.0070(train)	|	Acc: 84.2%(train)
	Loss: 0.0006(valid)	|	Acc: 76.4%(valid)
Epoch: 8  | time in 0 minutes, 6 seconds
	Loss: 0.0066(train)	|	Acc: 85.4%(train)
	Loss: 0.0006(valid)	|	Acc: 76.9%(valid)
Epoch: 9  | time